# RAGAs Evaluation

This notebook evaluates the RoundtableAI debate system using the RAGAs (Retrieval-Augmented Generation Assessment) framework.

**Metrics Evaluated:**
1. **Faithfulness**: Is the final answer grounded in the agent outputs (context)?
2. **Answer Relevancy**: Is the answer relevant to the question asked?

**Note:** We use only Faithfulness and Answer Relevancy because they don't require ground truth labels. Context Precision and Context Recall are skipped since:
- Backtesting already evaluates investment decision quality
- Tool selection already evaluates agent behavior correctness
- These two metrics assess response quality without needing reference answers

**Gemini Compatibility:**
- Answer Relevancy configured with `strictness=1` to avoid Gemini's "multiple candidates" error
- By default, Answer Relevancy generates 3 question variants, but Gemini doesn't support `candidate_count > 1`
- Using `strictness=1` generates only 1 question variant, making it compatible with Gemini

**Evaluation Setup:**
- System under test: Debate system (Gemini 2.0 Flash)
- Evaluator model: Google Gemini 3.0 Pro (independent evaluation)
- Test cases: 15 investment queries across different categories

## 1. Setup and Imports

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import json
import time
from datetime import datetime

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("Base imports successful")

Base imports successful


In [ ]:
# RAGAs imports
from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    # Not using these - they require ground truth:
    # ContextPrecision,
    # ContextRecall,
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset

print("RAGAs imports successful")

In [3]:
# Google Gemini imports for evaluation (different model from agents)
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Verify Google API key is set
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY not found in environment variables")

print("Google Gemini imports successful")

Google Gemini imports successful


In [4]:
# Import debate orchestrator
from agents.orchestrator import DebateOrchestrator

print("Orchestrator import successful")

Orchestrator import successful


## 2. Initialize Evaluator (Google Gemini 2.5 Pro)

In [ ]:
# Initialize Google Gemini 2.5 Pro for evaluation
# Using a different model than the agents (Gemini 2.0 Flash) for independent evaluation

evaluator_llm = ChatGoogleGenerativeAI(
    model="gemini-3-pro",
    temperature=0.0,  # Deterministic for evaluation
)

evaluator_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

# Wrap for RAGAs
ragas_llm = LangchainLLMWrapper(evaluator_llm)
ragas_embeddings = LangchainEmbeddingsWrapper(evaluator_embeddings)

print("Evaluator initialized: Gemini 3 Pro")
print("Embeddings: Google embedding-001")

Evaluator initialized: Gemini 2.5 Pro
Embeddings: Google embedding-001


## 3. Define Test Cases

Each test case contains:
- `query`: The investment question to ask the debate system
- `company`: The company/ticker being analyzed
- `category`: Type of query (comprehensive, fundamental, sentiment, valuation)

Note: We only use Faithfulness and Answer Relevancy metrics, which do NOT require ground truth answers.

## 3. Retrieve Database Data for Ground Truth Generation

To create proper ground truth answers, we need to query actual data from the database.

In [6]:
TEST_CASES = [
    # Comprehensive investment queries (primary use case)
    {
        "query": "Should I invest in Maybank?",
        "company": "Maybank",
        "category": "comprehensive"
    },
    {
        "query": "Is CIMB a good investment for a moderate risk investor?",
        "company": "CIMB",
        "category": "comprehensive"
    },
    {
        "query": "Analyze Public Bank for long-term investment",
        "company": "Public Bank",
        "category": "comprehensive"
    },
    {
        "query": "What is your recommendation for Tenaga Nasional?",
        "company": "Tenaga Nasional",
        "category": "comprehensive"
    },
    {
        "query": "Should I buy or sell Petronas Gas?",
        "company": "Petronas Gas",
        "category": "comprehensive"
    },
    
    # Risk-focused queries
    {
        "query": "Is IHH Healthcare safe for a conservative investor?",
        "company": "IHH Healthcare",
        "category": "risk_focused"
    },
    {
        "query": "Can I take an aggressive position on Press Metal?",
        "company": "Press Metal",
        "category": "risk_focused"
    },
    
    # Sector-specific queries
    {
        "query": "Evaluate Hong Leong Bank as a banking sector investment",
        "company": "Hong Leong Bank",
        "category": "sector_specific"
    },
    {
        "query": "Should I invest in Gamuda given infrastructure spending trends?",
        "company": "Gamuda",
        "category": "sector_specific"
    },
    
    # Growth vs Value queries
    {
        "query": "Is RHB Bank undervalued compared to its fundamentals?",
        "company": "RHB Bank",
        "category": "valuation_focused"
    },
    {
        "query": "Analyze Sunway's growth potential for my portfolio",
        "company": "Sunway",
        "category": "growth_focused"
    },
    
    # Dividend/Income queries
    {
        "query": "Is MISC Berhad good for dividend income?",
        "company": "MISC Berhad",
        "category": "income_focused"
    },
    
    # Comparative queries
    {
        "query": "Should I add Axiata to my telecommunications holdings?",
        "company": "Axiata",
        "category": "sector_allocation"
    },
    
    # Timing queries
    {
        "query": "Is now a good time to invest in YTL Corporation?",
        "company": "YTL Corporation",
        "category": "timing_focused"
    },
    
    # Stability queries  
    {
        "query": "How stable is Nestle Malaysia as an investment?",
        "company": "Nestle Malaysia",
        "category": "stability_focused"
    },
]

print(f"Defined {len(TEST_CASES)} test cases")
print("\nCategories:")
categories = {}
for tc in TEST_CASES:
    cat = tc["category"]
    categories[cat] = categories.get(cat, 0) + 1
for cat, count in categories.items():
    print(f"  {cat}: {count}")

Defined 15 test cases

Categories:
  comprehensive: 5
  risk_focused: 2
  sector_specific: 2
  valuation_focused: 1
  growth_focused: 1
  income_focused: 1
  sector_allocation: 1
  timing_focused: 1
  stability_focused: 1


## 4. Initialize Debate System

In [7]:
# Initialize the debate orchestrator
orchestrator = DebateOrchestrator(
    model_name="gemini-2.0-flash",
    max_rounds=5,
    consensus_threshold=0.75
)

print("Debate Orchestrator initialized")
print(f"  Model: gemini-2.0-flash")
print(f"  Max rounds: 5")
print(f"  Consensus threshold: 75%")

Initializing Gemini model: gemini-2.0-flash
Successfully connected to Gemini model: gemini-2.0-flash
Debate Orchestrator initialized
  Model: gemini-2.0-flash
  Max rounds: 5
  Consensus threshold: 75%


## 5. Run Debate and Extract Context

For each test case:
1. Run the debate system
2. Extract the context (agent analyses)
3. Get the final answer (recommendation + summary)

In [8]:
def run_debate_and_extract(orchestrator, query: str, company: str, risk_tolerance: str = "moderate") -> Dict[str, Any]:
    """
    Run a debate and extract the components needed for RAGAs evaluation.
    
    Returns:
        Dictionary with:
        - contexts: List of agent analyses (the retrieved/generated context)
        - answer: The final recommendation and summary
        - success: Whether the debate completed successfully
    """
    try:
        # Run the debate
        result = orchestrator.run_debate(company, risk_tolerance=risk_tolerance)
        
        # Extract contexts from agent messages
        contexts = []
        if result.debate_state:
            for msg in result.debate_state.messages:
                # Format each agent's analysis as context
                context_entry = f"[{msg.agent_type.upper()} AGENT - Round {msg.round_number}]\n{msg.content}"
                contexts.append(context_entry)
        
        # Construct the final answer
        answer = f"""Recommendation: {result.recommendation.value}
Confidence: {result.confidence:.0%}
Consensus Level: {result.consensus_level:.0%}

Summary:
{result.summary}

Key Points:
""" + "\n".join(f"- {point}" for point in result.key_points)
        
        if result.risks:
            answer += "\n\nRisks:\n" + "\n".join(f"- {risk}" for risk in result.risks)
        
        return {
            "success": True,
            "contexts": contexts,
            "answer": answer,
            "recommendation": result.recommendation.value,
            "confidence": result.confidence,
            "full_result": result
        }
        
    except Exception as e:
        import traceback
        return {
            "success": False,
            "error": str(e),
            "traceback": traceback.format_exc(),
            "contexts": [],
            "answer": ""
        }

print("Debate extraction function defined")

Debate extraction function defined


In [9]:
# Run debates for all test cases
# This will take some time due to API calls

debate_results = []
DELAY_BETWEEN_DEBATES = 5  # seconds

print("="*60)
print("RUNNING DEBATES FOR RAGAS EVALUATION")
print("="*60)

for i, test_case in enumerate(TEST_CASES):
    query = test_case["query"]
    company = test_case["company"]
    category = test_case["category"]
    
    print(f"\n[{i+1}/{len(TEST_CASES)}] {company} ({category})")
    print(f"  Query: {query}")
    
    # Determine risk tolerance from query
    risk_tolerance = "moderate"  # default
    if "conservative" in query.lower() or "safe" in query.lower():
        risk_tolerance = "conservative"
    elif "aggressive" in query.lower():
        risk_tolerance = "aggressive"
    
    # Run debate
    start_time = time.time()
    result = run_debate_and_extract(orchestrator, query, company, risk_tolerance)
    elapsed = time.time() - start_time
    
    if result["success"]:
        print(f"  Result: {result['recommendation']} (confidence: {result['confidence']:.0%})")
        print(f"  Contexts extracted: {len(result['contexts'])}")
        print(f"  Time: {elapsed:.1f}s")
    else:
        print(f"  ERROR: {result['error']}")
    
    debate_results.append({
        "test_case": test_case,
        "result": result
    })
    
    # Delay between debates to avoid rate limiting
    if i < len(TEST_CASES) - 1:
        time.sleep(DELAY_BETWEEN_DEBATES)

print("\n" + "="*60)
print(f"DEBATES COMPLETE: {sum(1 for r in debate_results if r['result']['success'])}/{len(TEST_CASES)} successful")
print("="*60)

RUNNING DEBATES FOR RAGAS EVALUATION

[1/15] Maybank (comprehensive)
  Query: Should I invest in Maybank?
[values] {'messages': [HumanMessage(content="Analyze MALAYAN BANKING BERHAD (1155.KL) from a MARKET SENTIMENT perspective.\n\n**INVESTOR PROFILE**: The investor is MODERATE - they seek a balance between growth and stability. They accept reasonable risk for better returns but want to avoid extreme volatility.\n\nYou are the Sentiment Analysis Agent. Your role is to evaluate market perception, news sentiment, and investor mood based on recent news articles and FinBERT sentiment scores.\n\n**CRITICAL: You MUST start your response with this EXACT structured header format:**\n```\n[DECISION]\nRECOMMENDATION: <BUY or HOLD or SELL>\nCONFIDENCE: <number from 0 to 100>%\n[/DECISION]\n```\n\nAfter the structured header, provide your detailed analysis.\n\nIn your analysis, cover:\n\n1. **Key Findings**: What does recent news sentiment reveal?\n   - Overall sentiment distribution (positive/neg

## 6. Prepare Data for RAGAs Evaluation

In [10]:
# Prepare data in RAGAs format
# Note: Only using Faithfulness and Answer Relevancy (no ground truth needed)
ragas_data = {
    "question": [],
    "answer": [],
    "contexts": [],
}

successful_cases = []

for dr in debate_results:
    if dr["result"]["success"] and dr["result"]["contexts"]:
        test_case = dr["test_case"]
        result = dr["result"]
        
        ragas_data["question"].append(test_case["query"])
        ragas_data["answer"].append(result["answer"])
        ragas_data["contexts"].append(result["contexts"])
        
        successful_cases.append(dr)

print(f"Prepared {len(ragas_data['question'])} cases for RAGAs evaluation")
print(f"\nSample data point:")
if ragas_data["question"]:
    print(f"  Question: {ragas_data['question'][0][:80]}...")
    print(f"  Answer length: {len(ragas_data['answer'][0])} chars")
    print(f"  Contexts: {len(ragas_data['contexts'][0])} entries")

Prepared 14 cases for RAGAs evaluation

Sample data point:
  Question: Should I invest in Maybank?...
  Answer length: 2887 chars
  Contexts: 6 entries


## 7. Run RAGAs Evaluation

**Rate Limiting Strategy:**
- Processes ONE test case at a time (not batch)
- Waits 15 seconds between evaluations
- Ensures we stay under Gemini's 25 requests/minute limit
- Total evaluation time: ~4-5 minutes for 14 test cases

In [11]:
# Create RAGAs dataset
dataset = Dataset.from_dict(ragas_data)

print(f"RAGAs Dataset created with {len(dataset)} samples")
print(f"\nColumns: {dataset.column_names}")

RAGAs Dataset created with 14 samples

Columns: ['question', 'answer', 'contexts']


In [ ]:
# Define metrics to evaluate
# Using only metrics that don't require ground truth
# NOTE: Configure answer_relevancy with strictness=1 to avoid Gemini's "multiple candidates" error
metrics = [
    Faithfulness(),                    # Does the answer follow from the context (agent outputs)?
    AnswerRelevancy(strictness=1),     # Does the answer address the question asked? (strictness=1 for Gemini compatibility)
]

print("Metrics to evaluate (no ground truth required):")
for m in metrics:
    print(f"  - {m.name}")
print("\nNote: answer_relevancy configured with strictness=1 for Gemini compatibility")

In [ ]:
# Run RAGAs evaluation with rate limiting (one at a time)
print("\n" + "="*60)
print("RUNNING RAGAS EVALUATION (One at a Time)")
print("Evaluator: Gemini 3 Pro")
print("="*60)

import time

all_scores = []
DELAY_SECONDS = 15  # Wait 15 seconds between evaluations to stay under 25 req/min

start_time = time.time()

for i in range(len(dataset)):
    print(f"\n[{i+1}/{len(dataset)}] Evaluating: {ragas_data['question'][i][:60]}...")
    
    # Create single-item dataset
    single_case = Dataset.from_dict({
        "question": [ragas_data["question"][i]],
        "answer": [ragas_data["answer"][i]],
        "contexts": [ragas_data["contexts"][i]],
    })
    
    # Evaluate single case
    case_start = time.time()
    result = evaluate(
        dataset=single_case,
        metrics=metrics,
        llm=ragas_llm,
        embeddings=ragas_embeddings,
    )
    case_elapsed = time.time() - case_start
    
    # Extract scores (result is a Dataset, so we need to access the first element)
    result_df = result.to_pandas()
    scores = {
        "faithfulness": result_df["faithfulness"].iloc[0],
        "answer_relevancy": result_df["answer_relevancy"].iloc[0],
    }
    all_scores.append(scores)
    
    print(f"  Faithfulness: {scores['faithfulness']:.3f}")
    print(f"  Answer Relevancy: {scores['answer_relevancy']:.3f}")
    print(f"  Time: {case_elapsed:.1f}s")
    
    # Wait between evaluations to respect rate limit (except for last case)
    if i < len(dataset) - 1:
        print(f"  Waiting {DELAY_SECONDS}s for rate limit...")
        time.sleep(DELAY_SECONDS)

# Combine results into RAGAs format
ragas_results = Dataset.from_dict({
    "question": ragas_data["question"],
    "answer": ragas_data["answer"],
    "contexts": ragas_data["contexts"],
    "faithfulness": [s["faithfulness"] for s in all_scores],
    "answer_relevancy": [s["answer_relevancy"] for s in all_scores],
})

elapsed = time.time() - start_time
print(f"\n" + "="*60)
print(f"Evaluation complete in {elapsed:.1f}s")
print(f"Average per case: {elapsed/len(dataset):.1f}s")
print("="*60)

## 8. Results Analysis

In [15]:
# Convert results to DataFrame
results_df = ragas_results.to_pandas()

# Add test case metadata
results_df["company"] = [sc["test_case"]["company"] for sc in successful_cases]
results_df["category"] = [sc["test_case"]["category"] for sc in successful_cases]

print("\n" + "="*70)
print("RAGAS EVALUATION RESULTS")
print("="*70)

# Display results
display_cols = ["company", "faithfulness", "answer_relevancy"]
print(results_df[display_cols].to_string(index=False))


RAGAS EVALUATION RESULTS
        company  faithfulness  answer_relevancy
        Maybank           NaN               NaN
           CIMB           NaN               NaN
    Public Bank           NaN               NaN
Tenaga Nasional           NaN               NaN
   Petronas Gas           NaN               NaN
 IHH Healthcare           NaN               NaN
    Press Metal           NaN               NaN
Hong Leong Bank           NaN               NaN
         Gamuda           NaN               NaN
       RHB Bank           NaN               NaN
    MISC Berhad           NaN               NaN
         Axiata           NaN               NaN
YTL Corporation           NaN               NaN
Nestle Malaysia           NaN               NaN


In [ ]:
# Calculate aggregate metrics
print("\n" + "="*70)
print("AGGREGATE METRICS")
print("="*70)

metrics_summary = {
    "Faithfulness": results_df["faithfulness"].mean(),
    "Answer Relevancy": results_df["answer_relevancy"].mean(),
}

for metric, score in metrics_summary.items():
    print(f"  {metric:20s}: {score:.2%}")

# Overall average
overall_avg = np.mean(list(metrics_summary.values()))
print(f"\n  {'Overall Average':20s}: {overall_avg:.2%}")

In [ ]:
# Metrics by category
print("\n" + "="*70)
print("METRICS BY CATEGORY")
print("="*70)

category_metrics = results_df.groupby("category").agg({
    "faithfulness": "mean",
    "answer_relevancy": "mean",
}).round(4)

print(category_metrics.to_string())

## 9. Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Overall metrics bar chart
ax1 = axes[0]
metrics_names = list(metrics_summary.keys())
metrics_values = list(metrics_summary.values())
colors = ['#3498db', '#2ecc71']

bars = ax1.bar(metrics_names, metrics_values, color=colors)
ax1.set_ylabel('Score')
ax1.set_title('RAGAs Metrics - Overall Performance')
ax1.set_ylim(0, 1.1)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.3)

# Add value labels on bars
for bar, val in zip(bars, metrics_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{val:.1%}', ha='center', va='bottom', fontsize=10)

# Plot 2: Per-company heatmap
ax2 = axes[1]
heatmap_data = results_df[["faithfulness", "answer_relevancy"]].values
companies = results_df["company"].values

im = ax2.imshow(heatmap_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax2.set_xticks(range(2))
ax2.set_xticklabels(['Faithfulness', 'Relevancy'], rotation=45, ha='right')
ax2.set_yticks(range(len(companies)))
ax2.set_yticklabels(companies, fontsize=8)
ax2.set_title('RAGAs Metrics by Company')

# Add colorbar
cbar = plt.colorbar(im, ax=ax2)
cbar.set_label('Score')

plt.tight_layout()
plt.savefig('ragas_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nChart saved to ragas_evaluation.png")

## 10. Export Results

In [ ]:
# Export detailed results to CSV
export_df = results_df[["company", "category", "question", "faithfulness", "answer_relevancy"]]
export_df.to_csv("ragas_detailed_results.csv", index=False)

print("Detailed results saved to ragas_detailed_results.csv")

In [ ]:
# Export summary metrics to JSON
summary_export = {
    "evaluation_date": datetime.now().isoformat(),
    "evaluator_model": "gemini-2.5-pro",
    "system_model": "gemini-2.0-flash",
    "total_test_cases": len(TEST_CASES),
    "successful_evaluations": len(results_df),
    "metrics_used": ["faithfulness", "answer_relevancy"],
    "note": "Only using metrics that don't require ground truth",
    "aggregate_metrics": {
        "faithfulness": float(metrics_summary["Faithfulness"]),
        "answer_relevancy": float(metrics_summary["Answer Relevancy"]),
        "overall_average": float(overall_avg)
    },
    "per_category_metrics": category_metrics.to_dict(),
    "per_case_results": results_df[["company", "category", "faithfulness", 
                                     "answer_relevancy"]].to_dict(orient='records')
}

with open("ragas_metrics.json", "w") as f:
    json.dump(summary_export, f, indent=2)

print("Summary metrics saved to ragas_metrics.json")

## 11. Final Report

In [ ]:
print("\n" + "="*70)
print("RAGAS EVALUATION - FINAL REPORT")
print("="*70)

print(f"\nEvaluation Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"System Under Test: Debate Orchestrator (gemini-2.0-flash)")
print(f"Evaluator Model: Gemini 2.5 Pro")
print(f"Total Test Cases: {len(TEST_CASES)}")
print(f"Successful Evaluations: {len(results_df)}")

print("\n" + "-"*50)
print("AGGREGATE METRICS")
print("-"*50)
for metric, score in metrics_summary.items():
    # Add interpretation
    if score >= 0.8:
        quality = "Excellent"
    elif score >= 0.6:
        quality = "Good"
    elif score >= 0.4:
        quality = "Fair"
    else:
        quality = "Needs Improvement"
    print(f"  {metric:20s}: {score:.2%} ({quality})")

print(f"\n  {'Overall Average':20s}: {overall_avg:.2%}")

print("\n" + "-"*50)
print("METRIC INTERPRETATION")
print("-"*50)
print("  Faithfulness:       How well the answer is grounded in agent outputs")
print("  Answer Relevancy:   How relevant the answer is to the question")

print("\n" + "-"*50)
print("WHY THESE METRICS?")
print("-"*50)
print("  These metrics don't require ground truth labels.")
print("  - Backtesting already evaluates investment decisions")
print("  - Tool selection already evaluates agent behavior")
print("  - RAGAs evaluates response quality and faithfulness")

print("\n" + "-"*50)
print("FILES GENERATED")
print("-"*50)
print("  - ragas_evaluation.png")
print("  - ragas_detailed_results.csv")
print("  - ragas_metrics.json")

print("\n" + "="*70)